# IBM/Coursera - Applied Data Science 
## *Capstone Final Report*

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1.  <a href="#item1">Introduction: Problem Statement</a>

2.  <a href="#item2">Data</a>

3.  <a href="#item3">Methodology</a>

4.  <a href="#item4">Results</a>

5.  <a href="#item5">Discussion</a>  

6.  <a href="#item6">Conclusion</a>
    </font>
    </div>

## 1. Introduction: Problem Statement <a name="item1"></a>

This project seeks to provide recommendations to management of the company I work for on neighborhoods within Manhattan, New York City which may be suitable candidates for opening up a second office location.

Our current office is located at 4 Times Square, New York. Given the location close to such a landmark in New York City such as Times Square, the current office is perfectly located for public transit as it has access to several New York City subway stations which service a number of subway lines. This means that the office is convenient for transportation for a good representation of the employees who travel to the office from all 5 boroughs in New York City.

The current office is also located in close proximity to other venues that the staff frequent outside the office such as gyms, restaurants for breakfast and lunch meals or evening entertainment venues following a hard day’s work in the office.

The company is growing and may soon need to secure additional office space. This search should aim to find alternative locations within Manhattan that would offer the same amenities and conveniences that the current office provides.

In this project we will explore locations and New York City subway information to identify alternative neighborhoods that have a similar profile to the current office and offer up a recommendation for locating a second office in the future.


#### Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [ ]:
# !pip install folium

In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

# import math for some distance and geometric calculations
import math

print('Libraries imported.')

Libraries imported.


## 2. Data <a name="item2"></a>

### 2.1	New York Neighborhoods Data

We will use the New York neighborhood data file used in the IBM Data Science course labs that contains the 5 boroughs in New York City and the neighborhoods that exist in each borough as well as the latitude and longitude coordinates of each neighborhood. This available at the URL: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json and will be read as a JSON file into a pandas dataframe

#### Let's load the data

In [5]:
!wget -q -O 'newyork_data.json' https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs/newyork_data.json

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

print('Data downloaded!')

Data downloaded!


#### All the relevant data is in the features key, which is basically a list of the neighborhoods. So, we define a new variable that includes this data.

In [7]:
neighborhoods_data = newyork_data['features']

#### Tranform the data into a _pandas_ dataframe

The next task is essentially transforming this data of nested Python dictionaries into a _pandas_ dataframe. So let's start by creating an empty dataframe.

In [8]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

#### Then let's loop through the data and fill the dataframe one row at a time.

In [9]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

#### Examine the resulting dataframe.

In [10]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


#### Make sure that the dataset has all 5 boroughs and 306 neighborhoods.

In [11]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


#### To limit the exploration to neighborhoods in Manhattan slice the original dataframe and create a new dataframe of the only neighbourhoods containing in the borough of Manhattan.

In [12]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [13]:
# Check the shape of the new dataframe to see how many neighborhoods we will be dealing with for Manhattan
manhattan_data.shape

(40, 4)

### 2.2 New York Public Transit Data

Most of the company’s employees use the New York City subway for transportation to the office. The proximity of the office location to a number of subway lines will be ideal. Data on the location of subway station will be sourced from the New York Metropolitan Transportation Authority (MTA). This data is available on the MTA’s website at http://web.mta.info/developers/developer-data-terms.html#data. A URL to a file containing subway station data is available at this location and this file is available in .csv format and will be read directly into a pandas dataframe using the read_csv function. The URL is as follows: https://atisdata.s3.amazonaws.com/Station/Stations.csv.

#### Import subway station data from MTA data source

In [14]:
df_station = pd.read_csv('https://atisdata.s3.amazonaws.com/Station/Stations.csv')
df_station.head()

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Notes
0,1,1,R01,BMT,Astoria,Astoria-Ditmars Blvd,Q,N W,Elevated,40.775036,-73.912034,NaN,Manhattan,0,NaN
1,2,2,R03,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770258,-73.917843,Ditmars Blvd,Manhattan,1,NaN
2,3,3,R04,BMT,Astoria,30 Av,Q,N W,Elevated,40.766779,-73.921479,Astoria - Ditmars Blvd,Manhattan,0,NaN
3,4,4,R05,BMT,Astoria,Broadway,Q,N W,Elevated,40.761820,-73.925508,Astoria - Ditmars Blvd,Manhattan,0,NaN
4,5,5,R06,BMT,Astoria,36 Av,Q,N W,Elevated,40.756804,-73.929575,Astoria - Ditmars Blvd,Manhattan,0,NaN


#### For each row create a list of all the subway lines stopping at each station. Capture this list in a separate column called 'Subway Lines'

In [15]:
df_station['Subway Lines'] = df_station['Daytime Routes'].str.split(' ')
df_station.head()

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Notes,Subway Lines
0,1,1,R01,BMT,Astoria,Astoria-Ditmars Blvd,Q,N W,Elevated,40.775036,-73.912034,NaN,Manhattan,0,NaN,"[N, W]"
1,2,2,R03,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770258,-73.917843,Ditmars Blvd,Manhattan,1,NaN,"[N, W]"
2,3,3,R04,BMT,Astoria,30 Av,Q,N W,Elevated,40.766779,-73.921479,Astoria - Ditmars Blvd,Manhattan,0,NaN,"[N, W]"
3,4,4,R05,BMT,Astoria,Broadway,Q,N W,Elevated,40.761820,-73.925508,Astoria - Ditmars Blvd,Manhattan,0,NaN,"[N, W]"
4,5,5,R06,BMT,Astoria,36 Av,Q,N W,Elevated,40.756804,-73.929575,Astoria - Ditmars Blvd,Manhattan,0,NaN,"[N, W]"


#### Explore how many unique subway lines there are using the new 'Subway Lines' column 

In [16]:
nyc_subway_lines = [] # a list of all unique subway lines available on the NYC subway system

# loop through each row in order to identify all unique values contained in 'Subway Lines'
for list in df_station['Subway Lines']:
    
    # loop through each cell and add an unique values identified to list of NYC subway lines
    for i in range(len(list)):
        if list[i] not in nyc_subway_lines: 
            nyc_subway_lines.append(list[i])

# sort list of a subway lines            
nyc_subway_lines.sort()

# print the number of subway lines
print('There are {} lines on the New York City subway system'.format(len(nyc_subway_lines)))

# display the first 10 lines from the sorted list of NYC subway lines
nyc_subway_lines[0:10]

There are 24 lines on the New York City subway system


['1', '2', '3', '4', '5', '6', '7', 'A', 'B', 'C']

#### Since we are only working with neighborhoods in Manhattan let's filter the dataset to only leave Manhattan subway stations. 

In [17]:
# Manhattan is represented by 'M' in the 'Borough' column. 
df_station1 = df_station[df_station['Borough'] == 'M']

# Since only one value remain in the 'Borough' column we can get rid of this column
df_station1.drop('Borough', axis='columns', inplace=True)

# Display the modified dataframe
df_station1.head()

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label,ADA,ADA Notes,Subway Lines
6,7,613,R11,BMT,Astoria,Lexington Av/59 St,N W R,Subway,40.762660,-73.967258,Queens,Downtown & Brooklyn,0,NaN,"[N, W, R]"
7,8,8,R13,BMT,Astoria,5 Av/59 St,N W R,Subway,40.764811,-73.973347,Queens,Downtown & Brooklyn,0,NaN,"[N, W, R]"
8,9,9,R14,BMT,Broadway - Brighton,57 St-7 Av,N Q R W,Subway,40.764664,-73.980658,Uptown & Queens,Downtown & Brooklyn,0,NaN,"[N, Q, R, W]"
9,10,10,R15,BMT,Broadway - Brighton,49 St,N R W,Subway,40.759901,-73.984139,Uptown & Queens,Downtown & Brooklyn,2,Uptown only,"[N, R, W]"
10,11,611,R16,BMT,Broadway - Brighton,Times Sq-42 St,N Q R W,Subway,40.754672,-73.986754,Uptown & Queens,Downtown & Brooklyn,1,NaN,"[N, Q, R, W]"


In [18]:
# Update dataframe to only leave information on the Subway lines and Coordinates for each subway station

df_station1 = df_station1[['Stop Name', 'Subway Lines','GTFS Latitude','GTFS Longitude']]
df_station1.reset_index(inplace=True)
df_station1.drop('index', axis='columns', inplace=True)

df_station1.head()

,Stop Name,Subway Lines,GTFS Latitude,GTFS Longitude
0,Lexington Av/59 St,"[N, W, R]",40.762660,-73.967258
1,5 Av/59 St,"[N, W, R]",40.764811,-73.973347
2,57 St-7 Av,"[N, Q, R, W]",40.764664,-73.980658
3,49 St,"[N, R, W]",40.759901,-73.984139
4,Times Sq-42 St,"[N, Q, R, W]",40.754672,-73.986754


#### Create function to visualize locations on a map of New York City

In [19]:
def add_markers(latitude, longitude, marker, dframe, folium_map, mark_color, opacity, rad):
    
    for lat, lng, label in zip(dframe[latitude], dframe[longitude], dframe[marker]):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=rad,
            popup=label,
            color=mark_color,
            fill=True,
            fill_color=mark_color,
            fill_opacity=opacity,
            parse_html=False).add_to(folium_map)

#### Use geopy library to get the latitude and longitude values of Manhattan for creating the map of New York City

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below.

In [20]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


#### Visualize Manhattan and the neighborhoods in it as well as all the subway station stops

In [21]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add Manhattan neighborhood markers to map
add_markers('Latitude', 'Longitude', 'Neighborhood', manhattan_data, map_manhattan, 'blue', 0.7, 5)
    
# add subway station markers to map
add_markers('GTFS Latitude', 'GTFS Longitude', 'Stop Name', df_station1, map_manhattan, 'red', 0.1, 1.5) 

# display map
map_manhattan

### 2.3 Foursquare data to explore venues around New York neighborhoods

The Foursquare API will be used to explore venues surrounding the New York neighborhoods in the Manhattan neighborhoods from the neighborhoods from the above dataset

#### Define Foursquare Credentials and Version

In [22]:
# The code was removed by Watson Studio for sharing.

Your credentails:
CLIENT_ID: ************
CLIENT_SECRET:************


The follow function explores each neighbourhood by using its longitute and latitude to retrieve the top 100 places within a radius of 500.

The function sends a GET request and examine the results in JSON format and returns the relevant information on the nearby venues in a pandas dataframe

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Run the above function to get relevant information on the nearby venues for each neighborhood and create a new dataframe called `manhattan_venues`.

In [24]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

In [25]:
# check the size of the manhattan_venues dataframe from running the function above

print(manhattan_venues.shape)
manhattan_venues.head(10)

(3221, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
4,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
5,Marble Hill,40.876551,-73.91066,Starbucks,40.873755,-73.908613,Coffee Shop
6,Marble Hill,40.876551,-73.91066,Blink Fitness,40.877271,-73.905595,Gym
7,Marble Hill,40.876551,-73.91066,Land & Sea Restaurant,40.877885,-73.905873,Seafood Restaurant
8,Marble Hill,40.876551,-73.91066,Astral Fitness & Wellness Center,40.876705,-73.906372,Gym
9,Marble Hill,40.876551,-73.91066,TCR The Club of Riverdale,40.878628,-73.914568,Tennis Stadium


In [26]:
# Check how many venues were returned for each neighborhood

manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,76,76,76,76,76,76
Carnegie Hill,88,88,88,88,88,88
Central Harlem,48,48,48,48,48,48
Chelsea,100,100,100,100,100,100
Chinatown,100,100,100,100,100,100
Civic Center,100,100,100,100,100,100
Clinton,100,100,100,100,100,100
East Harlem,43,43,43,43,43,43
East Village,100,100,100,100,100,100


In [27]:
# Find out how many unique categories can be curated from all the returned venues

print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))

There are 332 uniques categories.


## 3. Methodology <a name="item3"></a>

We will use the Manhattan neighborhood data and the Foursquare location data to explore the most common venues in each neighborhood. We will also use the New York City subway data to identify the nearby subway stations and subway lines that are within 1 mile of each neighborhood. 1 mile was considered a reasonable walking distance to get to the neighborhood after disembarking at a given subway station. 

The information of each neighborhoods venues and accessibility via the subway will be used to cluster the neighborhoods using k-means clustering.

The current office is in the Midtown neighborhood of Manhattan. We will observe which cluster this neighborhood falls into and will review other neighborhoods in the same cluster and examine these for similiarities with the current office location. 

A recommendation will be made on the ideal office location for a possible future second office out of the neighborhoods in the same cluster as the current office based on the neighborhoods that most closely replicates the most important factors that the employees value about the existing office.

#### We first analyze which venues are most common in each neighborhood

In [28]:
# one hot encoding
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood'] 

# move neighborhood column to the first column
first_col = manhattan_onehot.pop('Neighborhood')
manhattan_onehot.insert(0, 'Neighborhood', first_col)

manhattan_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cha Chaan Teng,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,

In [29]:
# examine the new dataframe size.

manhattan_onehot.shape

(3221, 333)

In [30]:
# Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped.head(10)

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cha Chaan Teng,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,

#### Load the grouped data into a pandas dataframe

First, create function to sort the venues in descending order.

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
# Create the new dataframe and display the top 10 venues for each neighborhood.

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(40)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Coffee Shop,Hotel,Park,Clothing Store,Gym,Memorial Site,Boat or Ferry,Food Court,Gourmet Shop,Burger Joint
1,Carnegie Hill,Coffee Shop,Pizza Place,Café,Bookstore,Gym / Fitness Center,Gym,Yoga Studio,Bar,Wine Shop,Shipping Store
2,Central Harlem,Public Art,French Restaurant,African Restaurant,American Restaurant,Fried Chicken Joint,Chinese Restaurant,Art Gallery,Bar,Gym / Fitness Center,Seafood Restaurant
3,Chelsea,Coffee Shop,Bakery,Art Gallery,French Restaurant,Italian Restaurant,Café,Ice Cream Shop,Wine Shop,American Restaurant,Cupcake Shop
4,Chinatown,Chinese Restaurant,Bakery,Cocktail Bar,American Restaurant,Spa,Optical Shop,Vietnamese Restaurant,Dessert Shop,Coffee Shop,Noodle House
5,Civic Center,Coffee Shop,Cocktail Bar,Spa,French Restaurant,Hotel,Gym / Fitness Center,American Restaurant,Park,Sushi Restaurant,Playground
6,Clinton,Gym / Fitness Center,Theater,Coffee Shop,American Restaurant,Italian Restaurant,Spa,Sandwich Place,Hotel,Cocktail Bar,Gym
7,East Harlem,Mexican Restaurant,Bakery,Deli / Bodega,Thai Restaurant,Pizza Place,Latin American Restaurant,Sandwich Place,French Restaurant,Spanish Restaurant,Grocery Store
8,East Village,Bar,Pizza Place,Mexican Restaurant,Cocktail Bar,Korean Restaurant,Wine Bar,Salon / Barbershop,Coffee Shop,Vegetarian / Vegan Restaurant,Ice Cream Shop
9,Financial District,Coffee Shop,Pizza Place,Cocktail Bar,Italian Restaurant,Mexican Restaurant,Café,American Restaurant,Gym,Park,Bar


#### Second, we use distances from each neighborhood to each station to determine how many subway stations and lines are in close proximity

To calculate distances in meters instead of degrees we calculate coordinates in the Cartesian 2D coordinate system The following functions convert between spherical coordinate system (latitude/longitude degrees) and cartesian coordinate system (X/Y coordinates in meters).

In [33]:
def latlon_to_xyz(lat,lon):
   
    r = 6371 # https://en.wikipedia.org/wiki/Earth_radius
    theta = math.pi/2 - math.radians(lat) 
    phi = math.radians(lon)
    x = r * math.sin(theta) * math.cos(phi) # bronstein (3.381a)
    y = r * math.sin(theta) * math.sin(phi)
    z = r * math.cos(theta)
    return [x,y,z]

def xyz_to_latlon (x,y,z):
    """Convert cartesian to angular lat/lon coordiantes"""
    r = math.sqrt(x**2 + y**2 + z**2)
    theta = math.asin(z/r) # https://stackoverflow.com/a/1185413/4933053
    phi = math.atan2(y,x)
    lat = math.degrees(theta)
    lon = math.degrees(phi)
    return [lat,lon]

In [34]:
# Test functions for converting between latitudes/longitudes and X/Y coordinates

print('Coordinate transformation check')
print('-------------------------------')
print('Manhattan center longitude={}, latitude={}'.format(longitude, latitude))
x, y, z = latlon_to_xyz(latitude, longitude)
print('Manhattan center UTM X={}, Y={}, Z={}'.format(x, y, z))
lo, la = xyz_to_latlon(x, y, z)
print('Manhattan center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
Manhattan center longitude=-73.9598939, latitude=40.7896239
Manhattan center UTM X=1332.8011884104667, Y=-4635.780619425009, Z=4162.069201819587
Manhattan center longitude=40.78962390000001, latitude=-73.9598939


#### Create function which adds a column for X and Y coordinates given the name of a dataframe and the columns containing the latitude and longitude coordinates

In [35]:
def add_xy_coordinates (dframe, latitude, longitude):
    xs = []
    ys = []

    for i in range(dframe.shape[0]):
        lat = dframe[latitude][i]
        lon = dframe[longitude][i]
        x, y, z = latlon_to_xyz(lat, lon)
        xs.append(x)
        ys.append(y)

    dframe['X'] = xs
    dframe['Y'] = ys

In [36]:
# Use function to add X/Y coordinates to the subsway stations dataframe

add_xy_coordinates(df_station1, 'GTFS Latitude', 'GTFS Longitude')

df_station1.head()

,Stop Name,Subway Lines,GTFS Latitude,GTFS Longitude,X,Y
0,Lexington Av/59 St,"[N, W, R]",40.762660,-73.967258,1332.746172,-4637.833890
1,5 Av/59 St,"[N, W, R]",40.764811,-73.973347,1332.210171,-4637.825398
2,57 St-7 Av,"[N, Q, R, W]",40.764664,-73.980658,1331.621314,-4638.005611
3,49 St,"[N, R, W]",40.759901,-73.984139,1331.434938,-4638.418884
4,Times Sq-42 St,"[N, Q, R, W]",40.754672,-73.986754,1331.327953,-4638.844513


In [37]:
# Use function to add X/Y coordinates to the manhattan_venues dataframe

add_xy_coordinates(manhattan_venues, 'Neighborhood Latitude', 'Neighborhood Longitude')

manhattan_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,X,Y
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place,1335.032701,-4628.56111
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio,1335.032701,-4628.56111
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner,1335.032701,-4628.56111
3,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop,1335.032701,-4628.56111
4,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop,1335.032701,-4628.56111


In [38]:
lat = []
lon = []

for i in range(manhattan_grouped.shape[0]):
    latitude = manhattan_venues.groupby('Neighborhood').mean()['Neighborhood Latitude'][i]
    longitude = manhattan_venues.groupby('Neighborhood').mean()['Neighborhood Longitude'][i]
    lat.append(latitude)
    lon.append(longitude)

manhattan_grouped['Latitude'] = lat
manhattan_grouped['Longitude'] = lon

add_xy_coordinates(manhattan_grouped, 'Latitude', 'Longitude')

manhattan_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cha Chaan Teng,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,

#### Loop through each neighborhood and determine the number of nearby station and the number of subway lines accessible through those stations

In [39]:
no_of_nearby_stations = []
no_of_nearby_subwaylines = []

for i in range(manhattan_grouped.shape[0]):
    
    stations_within_1m = []
    
    for j in range(df_station1.shape[0]):
        distance_to_station = math.sqrt((manhattan_grouped['X'][i] - df_station1['X'][j])**2 
                                        + (manhattan_grouped['Y'][i] - df_station1['Y'][j])**2)
              
        if distance_to_station < 0.75:
            stations_within_1m.append(df_station1['Subway Lines'][j])
        
        nearby_subway_line = []
        
        for list in stations_within_1m:
            # loop through each nearby station and add any unique values of NYC subway lines
            for k in range(len(list)):
                if list[k] not in nearby_subway_line: 
                    nearby_subway_line.append(list[k])
            
    
    no_of_nearby_stations.append(len(stations_within_1m))
    no_of_nearby_subwaylines.append(len(nearby_subway_line))

print(no_of_nearby_stations)
print(no_of_nearby_subwaylines) 

[13, 4, 6, 8, 14, 21, 3, 4, 3, 20, 17, 7, 17, 5, 2, 5, 4, 4, 16, 2, 6, 3, 2, 20, 19, 5, 7, 12, 1, 19, 1, 6, 11, 3, 3, 4, 5, 5, 8, 2]
[12, 4, 6, 9, 11, 15, 4, 2, 2, 13, 13, 6, 15, 5, 4, 2, 3, 7, 16, 4, 5, 5, 1, 19, 17, 3, 9, 10, 1, 17, 1, 10, 8, 4, 4, 4, 5, 2, 11, 1]


In [40]:
# add columns for the number of nearby stations and subway lines for each neighborhood
manhattan_grouped['Nearby Stations'] = no_of_nearby_stations
manhattan_grouped['Nearby Lines'] = no_of_nearby_subwaylines

# normalize the columns for numbers of stations and lines using MinMaxScaler fro the sklearn package
scaler = MinMaxScaler() 
scaled_values = scaler.fit_transform(manhattan_grouped[['Nearby Stations','Nearby Lines']]) 
manhattan_grouped[['Nearby Stations','Nearby Lines']] = scaled_values


#drop columns for longitude/latitude and X/Y coordinates as they are not needed for clustering
manhattan_grouped.drop(['Latitude', 'Longitude', 'X', 'Y'], axis='columns', inplace=True)

manhattan_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Austrian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cha Chaan Teng,Cheese Shop,Chinese Restaurant,Chocolate Shop,Christmas Market,Circus,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Arts Building,College Bookstore,College Cafeteria,Comedy Club,Community Center,Concert Hall,Convenience Store,Cooking School,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Drugstore,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,High School,Hill,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Latin American Restaurant,Laundry Service,Leather Goods Store,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music School,Music Venue,Nail Salon,New American Restaurant,Newsstand,Nightclub,Non-Profit,Noodle House,North Indian Restaurant,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Service,Pet Store,Pharmacy,Photography Studio,Physical Therapist,Piano Bar,Pie Shop,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Seafood Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skate Park,

#### Now that we have all the data on the venues and nearby subway stations and lines for each neighboorhood, we can now cluster the neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.


In [41]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:40] 

array([0, 1, 1, 2, 0, 3, 1, 1, 1, 3, 3, 1, 3, 1, 1, 1, 1, 1, 3, 1, 1, 1,
       4, 3, 3, 1, 2, 0, 4, 3, 4, 2, 2, 1, 1, 1, 1, 1, 2, 4], dtype=int32)

In [42]:
# Create a new dataframe that includes the cluster as well as the top 10 venues, 
# and the nearby stations and subway lines for each neighborhood and .

# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# merge manhattan_grouped with manhattan_data to add number of nearby stations and subway lines for each neighborhood
manhattan_grouped['Nearby Stations'] = no_of_nearby_stations
manhattan_grouped['Nearby Lines'] = no_of_nearby_subwaylines
manhattan_merged = manhattan_merged.join(manhattan_grouped[['Neighborhood', 'Nearby Stations','Nearby Lines']].set_index('Neighborhood'), on='Neighborhood')

# display updated columns
manhattan_merged.head() 

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Nearby Stations,Nearby Lines
0,Manhattan,Marble Hill,40.876551,-73.910660,4,Gym,Sandwich Place,Coffee Shop,Discount Store,Yoga Studio,Diner,Seafood Restaurant,Tennis Stadium,Supplement Shop,Donut Shop,2,1
1,Manhattan,Chinatown,40.715618,-73.994279,0,Chinese Restaurant,Bakery,Cocktail Bar,American Restaurant,Spa,Optical Shop,Vietnamese Restaurant,Dessert Shop,Coffee Shop,Noodle House,14,11
2,Manhattan,Washington Heights,40.851903,-73.936900,1,Café,Bakery,Grocery Store,Mobile Phone Shop,Chinese Restaurant,Shoe Store,Sandwich Place,Tapas Restaurant,Coffee Shop,Bank,5,2
3,Manhattan,Inwood,40.867684,-73.921210,1,Lounge,Mexican Restaurant,Café,Restaurant,Park,Bakery,Spanish Restaurant,Caribbean Restaurant,Chinese Restaurant,Frozen Yogurt Shop,5,2
4,Manhattan,Hamilton Heights,40.823604,-73.949688,1,Pizza Place,Coffee Shop,Café,Mexican Restaurant,Deli / Bodega,Yoga Studio,Latin American Restaurant,Liquor Store,Sushi Restaurant,Park,5,5


## 4. Results <a name="item4"></a>

We visualize the resulting clusters and examine each cluster and determine the discriminating venue categories that distinguish each cluster

In [43]:
# Visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [44]:
# first remove columns not considered necessary for this categorization

manhattan_merged.drop(['Borough','Latitude','Longitude'], axis='columns', inplace=True)

In [45]:
def print_subway_sumary(cluster):
    
    average_lines = int(manhattan_merged.loc[manhattan_merged['Cluster Labels'] == cluster]['Nearby Lines'].mean())
    min_no_station = manhattan_merged.loc[manhattan_merged['Cluster Labels'] == cluster]['Nearby Stations'].min()
    max_no_station = manhattan_merged.loc[manhattan_merged['Cluster Labels'] == cluster]['Nearby Stations'].max()

    print_statement = '\n Cluster {} neighborhoods have an average of {} line(s) accessible through {} to {} nearby stations'

    print(print_statement.format(cluster + 1, average_lines, min_no_station,max_no_station))

#### Cluster 1

In [46]:
print_subway_sumary(0)

manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0]


 Cluster 1 neighborhoods have an average of 11 line(s) accessible through 12 to 14 nearby stations


,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Nearby Stations,Nearby Lines
1,Chinatown,0,Chinese Restaurant,Bakery,Cocktail Bar,American Restaurant,Spa,Optical Shop,Vietnamese Restaurant,Dessert Shop,Coffee Shop,Noodle House,14,11
28,Battery Park City,0,Coffee Shop,Hotel,Park,Clothing Store,Gym,Memorial Site,Boat or Ferry,Food Court,Gourmet Shop,Burger Joint,13,12
31,Noho,0,Italian Restaurant,Coffee Shop,Cocktail Bar,Yoga Studio,Grocery Store,Sandwich Place,Art Gallery,Mexican Restaurant,Sushi Restaurant,Pizza Place,12,10


#### Cluster 2

In [47]:
print_subway_sumary(1)

manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1]


 Cluster 2 neighborhoods have an average of 4 line(s) accessible through 2 to 7 nearby stations


,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Nearby Stations,Nearby Lines
2,Washington Heights,1,Café,Bakery,Grocery Store,Mobile Phone Shop,Chinese Restaurant,Shoe Store,Sandwich Place,Tapas Restaurant,Coffee Shop,Bank,5,2
3,Inwood,1,Lounge,Mexican Restaurant,Café,Restaurant,Park,Bakery,Spanish Restaurant,Caribbean Restaurant,Chinese Restaurant,Frozen Yogurt Shop,5,2
4,Hamilton Heights,1,Pizza Place,Coffee Shop,Café,Mexican Restaurant,Deli / Bodega,Yoga Studio,Latin American Restaurant,Liquor Store,Sushi Restaurant,Park,5,5
5,Manhattanville,1,Coffee Shop,Deli / Bodega,Italian Restaurant,Mexican Restaurant,Seafood Restaurant,Cuban Restaurant,Sushi Restaurant,Supermarket,Burger Joint,Spanish Restaurant,3,5
6,Central Harlem,1,Public Art,French Restaurant,African Restaurant,American Restaurant,Fried Chicken Joint,Chinese Restaurant,Art Gallery,Bar,Gym / Fitness Center,Seafood Restaurant,6,6
7,East Harlem,1,Mexican Restaurant,Bakery,Deli / Bodega,Thai Restaurant,Pizza Place,Latin American Restaurant,Sandwich Place,French Restaurant,Spanish Restaurant,Grocery Store,4,2
8,Upper East Side,1,Italian Restaurant,Coffee Shop,Exhibit,Bakery,Boutique,Yoga Studio,Gym / Fitness Center,Pizza Place,French Restaurant,Spa,4,4
10,Lenox Hill,1,Italian Restaurant,Pizza Place,Sushi Restaurant,Coffee Shop,Café,Cocktail Bar,Burger Joint,Gym,Deli / Bodega,Gym / Fitness Center,4,3
12,Upper West Side,1,Italian Restaurant,Bar,French Restaurant,Wine Bar,Bakery,Café,Indian Restaurant,Coffee Shop,Yoga Studio,Seafood Restaurant,5,5
13,Lincoln Square,1,Café,Plaza,Performing Arts Venue,Theater,Italian Restaurant,Concert Hall,French Restaurant,Wine Shop,Gym / Fitness Center,Indie Movie Theater,4,7


#### Cluster 3

In [48]:
print_subway_sumary(2)

manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2]


 Cluster 3 neighborhoods have an average of 9 line(s) accessible through 6 to 11 nearby stations


,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Nearby Stations,Nearby Lines
16,Murray Hill,2,Coffee Shop,Hotel,Japanese Restaurant,Bar,Sandwich Place,Italian Restaurant,Burger Joint,American Restaurant,Gym / Fitness Center,Café,7,9
17,Chelsea,2,Coffee Shop,Bakery,Art Gallery,French Restaurant,Italian Restaurant,Café,Ice Cream Shop,Wine Shop,American Restaurant,Cupcake Shop,8,9
21,Tribeca,2,Park,American Restaurant,Italian Restaurant,Spa,Coffee Shop,Greek Restaurant,Wine Bar,Bakery,Men's Store,Café,11,8
24,West Village,2,Italian Restaurant,New American Restaurant,Cocktail Bar,American Restaurant,Jazz Club,Park,Theater,Wine Bar,Coffee Shop,Cosmetics Shop,8,11
34,Sutton Place,2,Italian Restaurant,Gym,Gym / Fitness Center,Pizza Place,Coffee Shop,Burger Joint,Furniture / Home Store,Park,Latin American Restaurant,Beer Garden,6,10


#### Cluster 4

In [49]:
print_subway_sumary(3)

manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3]


 Cluster 4 neighborhoods have an average of 15 line(s) accessible through 16 to 21 nearby stations


,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Nearby Stations,Nearby Lines
15,Midtown,3,Hotel,Clothing Store,Theater,Sporting Goods Shop,Coffee Shop,Bookstore,Bakery,Tailor Shop,Deli / Bodega,Cycle Studio,20,19
18,Greenwich Village,3,Italian Restaurant,Sushi Restaurant,Café,Clothing Store,Indian Restaurant,American Restaurant,Dessert Shop,Chinese Restaurant,Gym,Gourmet Shop,17,15
22,Little Italy,3,Bakery,Italian Restaurant,Cocktail Bar,Chinese Restaurant,Café,Bubble Tea Shop,Mediterranean Restaurant,Coffee Shop,Clothing Store,Pizza Place,16,16
23,Soho,3,Clothing Store,Italian Restaurant,Coffee Shop,Asian Restaurant,Mediterranean Restaurant,Bakery,Sporting Goods Shop,Boutique,Pizza Place,Gym,19,17
29,Financial District,3,Coffee Shop,Pizza Place,Cocktail Bar,Italian Restaurant,Mexican Restaurant,Café,American Restaurant,Gym,Park,Bar,20,13
32,Civic Center,3,Coffee Shop,Cocktail Bar,Spa,French Restaurant,Hotel,Gym / Fitness Center,American Restaurant,Park,Sushi Restaurant,Playground,21,15
33,Midtown South,3,Korean Restaurant,Hotel,Japanese Restaurant,American Restaurant,Dessert Shop,Salad Place,Café,Gym / Fitness Center,Coffee Shop,Scenic Lookout,19,17
38,Flatiron,3,Coffee Shop,American Restaurant,Japanese Restaurant,New American Restaurant,Spa,Sporting Goods Shop,Mexican Restaurant,Mediterranean Restaurant,Italian Restaurant,Gym / Fitness Center,17,13


#### Cluster 5

In [50]:
print_subway_sumary(4)

manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4]


 Cluster 5 neighborhoods have an average of 1 line(s) accessible through 1 to 2 nearby stations


,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Nearby Stations,Nearby Lines
0,Marble Hill,4,Gym,Sandwich Place,Coffee Shop,Discount Store,Yoga Studio,Diner,Seafood Restaurant,Tennis Stadium,Supplement Shop,Donut Shop,2,1
9,Yorkville,4,Italian Restaurant,Coffee Shop,Gym,Bar,Deli / Bodega,Japanese Restaurant,Diner,Wine Shop,Sushi Restaurant,Pizza Place,2,1
11,Roosevelt Island,4,Park,Japanese Restaurant,School,Liquor Store,Greek Restaurant,Metro Station,Supermarket,Bubble Tea Shop,Bus Line,Food & Drink Shop,1,1
37,Stuyvesant Town,4,Park,Bar,Coffee Shop,Baseball Field,Pet Service,Gas Station,Boat or Ferry,Skating Rink,Bistro,Farmers Market,1,1


### Results Summary: Observations

<b>Cluster 1</b>: The following similarities were observed for neighborhoods in this cluster:
* The neighborhoods are generally residential neighborhoods in the surrounding Midtown and northern sections of Downtown Manhattan
* Coffe shops and Italian restaurants are very prevalent in this cluster
* Neighborhoods have an average of 9 lines accessible through 6 to 8 nearby stations


<b>Cluster 2</b>: The following similarities were observed for neighborhoods in this cluster:
* The neighborhoods are located close to the center of the Manhattan island when considered from a west to east position
* Their central location means that several subway lines traverse through these neighborhood making them the most accessible from a public transit perspective:
    * Neighborhoods have an average of 15 lines accessible through 16 to 21 nearby stations
* The neighborhoods have a wide variety of most popular venues which makes them suitable for catering for a wide range of needs

<b>Cluster 3</b>: The following similarities were observed for neighborhoods in this cluster:
* The neighborhoods are generally residential neighborhoods on the eastern side of the Manhattan
* The neighborhoods have poor access to tranportation with fewer stations and subway lines servicing these locations:
    * Neighborhoods have an average of 1 line accessible through 1 to 5 nearby stations
* Restaurants are less prevalent in this cluster relative to other clusters and are instead replaced by supermarkets, deli/bodegas, markets and other residential neighborhood businesses


<b>Cluster 4</b>: The following similarities were observed for neighborhoods in this cluster:
* The neighborhoods are generally residential neighborhoods surrounding Central Park
* The neighborhoods have limited access to tranportation although it is not as poor as cluster 3:
    * Neighborhoods have an average of 4 line accessible through 2 to 6 nearby stations
* This is the cluster with the most neighborhoods

<b>Cluster 5</b>: The following similarities were observed for neighborhoods in this cluster:
* The neighborhoods are generally residential neighborhoods in Downtown Manhattan
* The neighborhoods have the second best acess to subway tranportation: neighborhoods have an average of 10 line accessible through 11 to 14 nearby stations
* This is the cluster with the least neighborhoods

## 5.  Discussion <a name="item5"></a>

Our approach was to focus on the cluster into which the current office location falls. The idea was that neighborhoods clustered together with the present location would be viable candidates for a possible second office. The current office is located at 4 Times Square. This neighborhood is located in the Midtown neighborhood. The Midtwon neighborhood was categorized in Cluster 2. We will therefore discuss Cluster 2 (cluster label 1) in more detail in arriving at a suitable recommendation for a second office location.

In [52]:
# A recap of cluster 2:

manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Nearby Stations,Nearby Lines
2,Washington Heights,1,Café,Bakery,Grocery Store,Mobile Phone Shop,Chinese Restaurant,Shoe Store,Sandwich Place,Tapas Restaurant,Coffee Shop,Bank,5,2
3,Inwood,1,Lounge,Mexican Restaurant,Café,Restaurant,Park,Bakery,Spanish Restaurant,Caribbean Restaurant,Chinese Restaurant,Frozen Yogurt Shop,5,2
4,Hamilton Heights,1,Pizza Place,Coffee Shop,Café,Mexican Restaurant,Deli / Bodega,Yoga Studio,Latin American Restaurant,Liquor Store,Sushi Restaurant,Park,5,5
5,Manhattanville,1,Coffee Shop,Deli / Bodega,Italian Restaurant,Mexican Restaurant,Seafood Restaurant,Cuban Restaurant,Sushi Restaurant,Supermarket,Burger Joint,Spanish Restaurant,3,5
6,Central Harlem,1,Public Art,French Restaurant,African Restaurant,American Restaurant,Fried Chicken Joint,Chinese Restaurant,Art Gallery,Bar,Gym / Fitness Center,Seafood Restaurant,6,6
7,East Harlem,1,Mexican Restaurant,Bakery,Deli / Bodega,Thai Restaurant,Pizza Place,Latin American Restaurant,Sandwich Place,French Restaurant,Spanish Restaurant,Grocery Store,4,2
8,Upper East Side,1,Italian Restaurant,Coffee Shop,Exhibit,Bakery,Boutique,Yoga Studio,Gym / Fitness Center,Pizza Place,French Restaurant,Spa,4,4
10,Lenox Hill,1,Italian Restaurant,Pizza Place,Sushi Restaurant,Coffee Shop,Café,Cocktail Bar,Burger Joint,Gym,Deli / Bodega,Gym / Fitness Center,4,3
12,Upper West Side,1,Italian Restaurant,Bar,French Restaurant,Wine Bar,Bakery,Café,Indian Restaurant,Coffee Shop,Yoga Studio,Seafood Restaurant,5,5
13,Lincoln Square,1,Café,Plaza,Performing Arts Venue,Theater,Italian Restaurant,Concert Hall,French Restaurant,Wine Shop,Gym / Fitness Center,Indie Movie Theater,4,7


A review of Cluster 2 shows that 8 neighborhoods were categorized into this cluster. We omit the Midtown neighborhood as this is the neighborhood of the current office location and it would not be advisable to have 2 offices located in the same neighborhood. To ensure better coverage of the island it would be more prudent to locate the second office location in a different neighborhood; preferably one that is some distance for the current location to achieve the best possible area coverage. On this basis we omit Midtown South and Flatiron neighborhoods as they are too close to the current office location.

This leaves 5 neighborhoods that are located in Downtown Manhattan:
* Greenwich Village
* Little Italy
* Soho
* Financial District
* Civic Center

Looking at the 5 remaining options we observe based on knowledge of Manhattan that 3 of the 5 remaining neighborhoods are primarily residential neighborhoods and would not be suitable for securing commercial real estate space. These neighborhoods are Greenwhich Village, Little Italy and Soho.

We are down to 2 possible candidates - Financial District and Civic Center.

As discussed earlier, subway transportation is an important consideration for making sure that the commutes of employees are as convenient as possible. We notice that the Civic Center has better subway access to the Financial District when measured by number of stations and subway lines that are nearby. Based on knowledge of Manhattan we also observe that rentals in the Financial District will be much more expensive as this is the area were Wall Stree is located and is highly sort after by financial services firm both locally and internationally.


## 6. Conclusion <a name="item6"></a>

After considering the neighborhoods that seem most similar to the current location we determine that the Civic Center neighborhood should be considered as the best option for locating a second office location. This determination was made based on the following factors:
* The most comparable subway transportation access to the current Midtown office
* The more economic neighborhood relative to other commercial district that are most similar to the Midtown location
* This neighborhood is in Downtown Manhattan which gives is good separation from the current Midtwon location and therefore achieves good area coverage
* Similarity in common venues relative to the current office. For example 4 out of the top 10 venues are similar

In conclusion we recommend that the second location be located in the Civic Center neighborhood of Manhattan, New York City.
